In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
%load_ext autoreload
%autoreload 2

adaptive softmax - https://medium.com/analytics-vidhya/adaptive-softmax-explained-in-numpy-941487ac4ba6

In [3]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm_notebook

In [4]:
import  datatset
from torch.utils.data import DataLoader

In [5]:
# DataLoader

In [6]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [7]:
label_counts  = df["category_id"].value_counts()

In [8]:
num_classes = len(label_counts) + 1

In [9]:
labels_ordered = label_counts.keys().tolist()

In [10]:
ds = datatset.HerbDataSet(df_train, TRAIN_PATH,128,label="category_id")

In [11]:
data_loader = DataLoader(ds, batch_size=16, shuffle=True)

In [12]:
# Model

In [13]:
# import model
import torchvision.models as models

In [14]:
import torch.nn as nn
import torch

In [15]:
class AdaptiveLogModel(nn.Module):
    def __init__(self, base, num_classes):
        super(AdaptiveLogModel, self).__init__()
        self.base_model = base
        self.base_model.fc = nn.Linear(in_features=base.fc.in_features, out_features=1024)
        self.adaptiveSofmax = nn.AdaptiveLogSoftmaxWithLoss(1024, num_classes, [500, 1000, 5000])
                     
    def forward(self, input, target):
        logits = self.base_model(input)
        return self.adaptiveSofmax(logits, target)
        
    

In [16]:
def test_model():
    base = models.resnet18(pretrained=True)
    base.fc  = nn.Linear(in_features=512, out_features=512)
    return base.cuda()


In [17]:
base = models.resnet18(pretrained=True)
model = AdaptiveLogModel(base, num_classes)
model.cuda()
1

1

In [31]:
# model = test_model()

In [32]:
# Tracking

In [33]:
from torch.utils.tensorboard import SummaryWriter
import metrics
from train import Training

In [34]:
writer = SummaryWriter(TRAIN_PATH/"logs/run-2")
acc =  metrics.Accuracy()
manager = metrics.MetricManager([acc], writer)

In [35]:
# Train

In [36]:
class Trainer(Training):
    def __init(self,  metrics, loss, optim, data, epochs, model, save_dir):
        super(self).__init__(self,  metrics, loss, optim, data, epochs, model, save_dir)
        
    def train_step(self, batch):
        data, labels = batch
        output = self.model(data, labels)
        
#         self.metrics.update(output.output, labels, self.step)
#         if self.metrics.writer:
#             self.metrics.writer.add_scalar("loss", output.loss.item(), self.step)
#             print(f'Loss: {output.loss.item()}')

        self.optim.zero_grad()  # zero gradients
        output.loss.backward()  # calculate gradients
        self.optim.step()  # updated weights


In [18]:
criterion = nn.CrossEntropyLoss()

In [19]:
optimizer = torch.optim.Adam(model.parameters())

In [20]:
# adaptiveSofmax = nn.AdaptiveLogSoftmaxWithLoss(512, num_classes, [500, 1000, 5000]).cuda()

In [ ]:
for batch in tqdm_notebook(data_loader):
        data, labels = batch
        output = model(data, labels)
#         output = adaptiveSofmax(logits, labels)
        optimizer.zero_grad()  # zero gradients
        output.loss.backward()  # calculate gradients
        optimizer.step()  # updated weights
    

In [ ]:
criterion = None
# criterion = nn.AdaptiveLogSoftmaxWithLoss(num_calsses, num_calsses, [50, 500, 5000])

In [ ]:
model_path=TRAIN_PATH/"models/"

In [ ]:
trainer = Trainer(manager, criterion, optimizer, data_loader, 3, model, model_path )

In [ ]:
trainer.train_loop()